##**モーション動画の環境構築**


In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
!pip install torch==2.1.0 torchvision==0.16.0 opencv-python git+https://github.com/facebookresearch/detectron2@main#subdirectory=projects/DensePose

Cloning into 'detectron2'...
remote: Enumerating objects: 15292, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 15292 (delta 3), reused 8 (delta 1), pack-reused 15275
Receiving objects: 100% (15292/15292), 6.18 MiB | 10.80 MiB/s, done.
Resolving deltas: 100% (11114/11114), done.
  Cloning https://github.com/facebookresearch/detectron2 (to revision main) to /tmp/pip-req-build-4t13hoeh
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2 /tmp/pip-req-build-4t13hoeh
  Resolved https://github.com/facebookresearch/detectron2 to commit a0e22dbfa791e6235e4f196d5ce25e754d02be31
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-nh4dmvet/detectron2_bdc76092ef7d4716b3a919638245e12a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-nh4dmvet

##**モーション動画の作成**
- モーション動画の元となる動画は正方形に加工してください。
- パスを変更してから実行してね。

In [ ]:
from detectron2.config import get_cfg
import torch
import cv2
import numpy as np
from detectron2.engine import DefaultPredictor
from densepose import add_densepose_config
from densepose.vis.extractor import DensePoseResultExtractor
from densepose.vis.densepose_results import DensePoseResultsFineSegmentationVisualizer as Visualizer

# Initialize Detectron2 configuration for DensePose
cfg = get_cfg()
add_densepose_config(cfg)
cfg.merge_from_file("detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml")
cfg.MODEL.WEIGHTS = "https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl"
predictor = DefaultPredictor(cfg)

# Video paths
INPUT_VIDEO_PATH = "/content/1207.mp4" #@param {type:"string"}
OUTPUT_VIDEO_PATH = "/content/output.mp4" #@param {type:"string"}

# Open the input video
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
fps = 25

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (512, 512))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (512, 512))
    with torch.no_grad():
        outputs = predictor(frame)['instances']

    results = DensePoseResultExtractor()(outputs)
    out_frame = Visualizer(alpha=1).visualize(np.zeros(frame.shape, dtype=np.uint8), results)

    out.write(out_frame)

# Release resources
cap.release()
out.release()

model_final_162be9.pkl: 256MB [00:01, 135MB/s]                           
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


##**MagicAnimateの立ち上げ**


In [ ]:
%cd /content
!git clone https://github.com/YutaAkiyama/magicanimate-hf

!pip install -q diffusers==0.21.4 transformers==4.32.0 accelerate==0.22.0 omegaconf==2.3.0 einops==0.6.1 av gradio
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

!apt -y install -qq aria2
BaseModelUrl = "https://huggingface.co/runwayml/stable-diffusion-v1-5"
BaseModelDir = "/content/magicanimate-hf/stable-diffusion-v1-5"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/model_index.json -d {BaseModelDir} -o model_index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/vae/diffusion_pytorch_model.bin -d {BaseModelDir}/vae -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/vae/config.json -d {BaseModelDir}/vae -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/unet/diffusion_pytorch_model.bin -d {BaseModelDir}/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/unet/config.json -d {BaseModelDir}/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/vocab.json -d {BaseModelDir}/tokenizer -o vocab.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/tokenizer_config.json -d {BaseModelDir}/tokenizer -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/special_tokens_map.json -d {BaseModelDir}/tokenizer -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/tokenizer/merges.txt -d {BaseModelDir}/tokenizer -o merges.txt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/text_encoder/pytorch_model.bin -d {BaseModelDir}/text_encoder -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/text_encoder/config.json -d {BaseModelDir}/text_encoder -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/scheduler/scheduler_config.json -d {BaseModelDir}/scheduler -o scheduler_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/resolve/main/safety_checker/pytorch_model.bin -d {BaseModelDir}/safety_checker -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/safety_checker/config.json -d {BaseModelDir}/safety_checker -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {BaseModelUrl}/raw/main/feature_extractor/preprocessor_config.json -d {BaseModelDir}/feature_extractor -o preprocessor_config.json

%cd /content/magicanimate-hf
!python app.py

Streaming output truncated to the last 5000 lines.
v1-5-pruned-emaonly.ckpt:  84% 3.60G/4.27G [03:38<00:49, 13.5MB/s]







v1-5-pruned.safetensors:  33% 2.57G/7.70G [02:20<06:51, 12.5MB/s]


v1-5-pruned-emaonly.safetensors:  86% 3.65G/4.27G [03:37<00:45, 13.5MB/s]




v1-5-pruned.ckpt:  37% 2.87G/7.70G [02:41<06:02, 13.3MB/s]

v1-5-pruned-emaonly.ckpt:  85% 3.61G/4.27G [03:38<00:40, 16.5MB/s]







v1-5-pruned.safetensors:  33% 2.58G/7.70G [02:20<05:25, 15.7MB/s]


v1-5-pruned-emaonly.safetensors:  86% 3.66G/4.27G [03:37<00:36, 16.5MB/s]

v1-5-pruned-emaonly.ckpt:  85% 3.62G/4.27G [03:38<00:32, 20.2MB/s]







v1-5-pruned.safetensors:  34% 2.59G/7.70G [02:20<04:26, 19.2MB/s]


v1-5-pruned-emaonly.safetensors:  86% 3.67G/4.27G [03:37<00:29, 20.3MB/s]

v1-5-pruned-emaonly.ckpt:  85% 3.63G/4.27G [03:38<00:24, 25.5MB/s]







v1-5-pruned.safetensors:  34% 2.60G/7.70G [02:20<03:28, 24.5MB/s]




v1-5-pruned.ckpt:  37% 2.88G/7.70G [02:41<05:21, 15.0MB/s]


v1-5-pruned-emaonly.safetensor